In [ ]:
import pandas as pd

In [ ]:
path = '/home/aiops/zhuty/tinyllama/scripts/fewshot_eval/results/rag_outputs_complete.csv'
df = pd.read_csv(path)
df.head()

In [ ]:
df

In [ ]:
selected_task = 'tq_obqa'
df = df[df['task'] == selected_task]
df = df[df['num_docs'] <=10]
print(len(df))

In [ ]:
def extract_pretrain_dataset(x):
    """
    Sample input: /home/aiops/zhuty/tinyllama/out/tiny_LLaMA_1b_8k_cc_8k/iter-200000-ckpt-step-25000_hf
    Sample output: cc_8k
    Args:
        x:
    Returns:

    """
    if '/home/aiops/zhuty' in x:
        folder_name = x.split('/')[-2] # tiny_LLaMA_1b_8k_cc_8k
        return ("_").join(folder_name.split('_')[4:]) # cc_8k
    else:
        # sample model name: tyzhu/tiny_LLaMA_120M_8k_cc_8k_iter-240000-ckpt-step-60000_hf
        return x.strip("tyzhu/").split("_8k_")[1]

name_to_meaning_map = {"cc_8k": "random", "intramask_cc_8k": "intradoc", "cc_merged_v1_8k": "dense", "cc_merged_v2_8k": "bm25"}

df['model'] = df['model'].apply(extract_pretrain_dataset)
df['model'] = df['model'].map(name_to_meaning_map)

In [ ]:
grouped_df = df.groupby(['model', 'task', 'num_docs' ]).agg({'score': ['mean', 'std']})

In [ ]:
grouped_df

In [ ]:
# visualize as a line plot
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 6))
# plot the mean and the standard deviation
for model in grouped_df.index.get_level_values(0).unique():
    for task in grouped_df.index.get_level_values(1).unique():
        df_to_plot = grouped_df.loc[(model, task)]
        plt.errorbar(df_to_plot.index.get_level_values(0), df_to_plot['score']['mean'], yerr=df_to_plot['score']['std'], label=model)
plt.title('RAG performance on {}'.format(selected_task.upper()))
plt.ylabel('Mean Score')
plt.xlabel('Number of documents')
plt.legend()
plt.show()